# YOLOv8n Resistor Detector — Colab Training Notebook

**Project:** VivaLaResistance  
**Dataset:** [isha-74mjj/yolov5-u3oks v3](https://universe.roboflow.com/isha-74mjj/yolov5-u3oks/dataset/3) on Roboflow (4,422 images, CC-BY 4.0)  
**Target:** YOLOv8n → ONNX opset 17, fixed batch=1, 640px input  
**Runtime:** Google Colab with T4 GPU (Runtime → Change runtime type → GPU)

---

## Section 1: Setup

Install `ultralytics` (YOLOv8) and `roboflow` (dataset download). This takes ~60 seconds on a fresh Colab runtime.

In [ ]:
# Install dependencies
!pip install ultralytics roboflow

import os
from ultralytics import YOLO
from roboflow import Roboflow

## Section 2: Download Dataset from Roboflow

You need a free Roboflow account. Get your API key at [roboflow.com](https://roboflow.com) → Settings → API.  
Replace `"YOUR_ROBOFLOW_API_KEY"` below before running.

In [ ]:
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")  # get free key at roboflow.com
project = rf.workspace("isha-74mjj").project("yolov5-u3oks")
version = project.version(3)  # confirmed v3 — https://universe.roboflow.com/isha-74mjj/yolov5-u3oks/dataset/3
dataset = version.download("yolov8")

## Section 3: Train YOLOv8n

Trains YOLOv8-nano on the resistor dataset with early stopping (patience=10).  
**With a T4 GPU on Colab, 50 epochs on 4,422 images takes ~25–40 minutes.**

In [ ]:
model = YOLO("yolov8n.pt")
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,          # GPU (Colab T4)
    patience=10,       # early stopping
    project="resistor-detector",
    name="yolov8n-run1",
    exist_ok=True
)

## Section 4: Evaluate

Run validation on the held-out val set to check model quality before exporting.

**Target: mAP50 > 0.70 before exporting. If below 0.65, increase epochs to 100.**

In [ ]:
metrics = model.val()
print(f"mAP50:    {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

## Section 5: Export to ONNX

Export the best checkpoint to ONNX format with fixed batch size 1 (required for mobile inference via `Microsoft.ML.OnnxRuntime`).

In [ ]:
model.export(
    format="onnx",
    imgsz=640,
    simplify=True,
    opset=17,
    dynamic=False      # fixed batch=1 for mobile
)
# Output: resistor-detector/yolov8n-run1/weights/best.onnx
print("Export complete. Download best.onnx from the Files panel →")

## Section 6: Validate the Exported Model

1. Download `best.onnx` from the Colab Files panel (folder icon on the left sidebar)
2. Open [https://netron.app](https://netron.app) and drag the file in
3. Verify:
   - **Input:** `float32 [1, 3, 640, 640]`
   - **Output:** `float32 [1, 5, 8400]`  
     *(YOLOv8 detection head — 4 bbox coords + 80 COCO classes, but we have 1 class so it will be `[1, 5, 8400]`)*
4. Rename file to `resistor-localization.onnx`
5. Place in: `src/VivaLaResistance/Resources/Raw/resistor-localization.onnx`
6. Set build action to `MauiAsset` in the `.csproj`

## Section 7: C# Integration Notes

### ONNX Output Format

YOLOv8 exports a single output tensor shaped `[1, 5, 8400]` where:
- **8400** = number of candidate anchor boxes (decoded, not raw anchors)
- **5 values per box** = `[cx, cy, w, h, conf]`
  - `cx`, `cy` — box centre, normalised to `[0, 1]` relative to 640×640 input
  - `w`, `h` — box dimensions, normalised to `[0, 1]`
  - `conf` — objectness confidence score `[0, 1]`

### Post-Processing (NMS)

ONNX Runtime does **not** apply NMS internally. You must apply it in C# after inference:
- **Confidence threshold:** 0.25 (discard boxes below this)
- **IoU threshold:** 0.45 (suppress overlapping boxes in NMS)

### Input Preprocessing

The MAUI camera frame arrives as **BGRA8888** byte[]. Convert to the model's expected format:

1. **Resize** to 640×640 (letterbox if aspect ratio differs, pad with grey `(114,114,114)`)
2. **Reorder channels:** BGRA → RGB (drop alpha, swap B and R)
3. **Normalise:** divide each pixel value by 255.0 → `float32` in `[0, 1]`
4. **CHW layout:** output a flat `float32[3 × 640 × 640]` array in planar order (all R, then all G, then all B)

### C# Stub

`OnnxResistorLocalizationService.cs` in `src/VivaLaResistance.Services/` is scaffolded and compiles against `Microsoft.ML.OnnxRuntime 1.20.1`. Once `resistor-localization.onnx` is placed in `Resources/Raw/` with `MauiAsset` build action, implement:
- `InitializeAsync` — load model from stream via `InferenceSession`
- `InferAsync` — preprocess BGRA8888 frame, run session, decode `[1, 5, 8400]` output, apply NMS, return `IReadOnlyList<ResistorBoundingBox>`

Bounding box coordinates returned to Shuri's overlay renderer are **normalised `[0, 1]`** as defined in `ResistorBoundingBox` (Core.Models).